In [7]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

## Task1

빈 칸을 채워주세요!

단계별 output이 github 파일에는 남아있으니 그 output과 동일한 형태인지 확인하면서 진행해주시면 됩니다~

In [213]:
#1. 생성할 문장 데이터

sentence = ("Brick walls are there for a reason and you must not think "
            "that the brick walls aren't there to keep us out, but rather "
            "in this way that the brick walls are there to show us how badly we want things.")

In [214]:
#2. 문자 집합 만들기
world_set = list(set(sentence))

## 문제(1): 각 문자에 정수 인코딩 (공백도 하나의 원소로 포함)
vocab = {chr:idx for idx, chr in enumerate(world_set)}

In [215]:
print(vocab)

{',': 0, 'm': 1, 'e': 2, 'h': 3, 'k': 4, ' ': 5, 'l': 6, 'p': 7, 'f': 8, 'c': 9, 'w': 10, 'r': 11, 'a': 12, 'o': 13, "'": 14, 't': 15, '.': 16, 'g': 17, 'd': 18, 'B': 19, 'n': 20, 'b': 21, 's': 22, 'y': 23, 'i': 24, 'u': 25}


In [216]:
#3. 문자 집합 크기 확인

vocab_size = len(vocab)
print('문자 집합 크기 : {}'.format(vocab_size))

문자 집합 크기 : 26


In [217]:
#4. 하이퍼 파라미터 설정(자유롭게 수정해보세요!)

hidden_size = vocab_size # 같아야 하는 것 확인!
sequence_length = 10  # 너무 길거나 너무 짧게 잡으면 안됩니다!
learning_rate = 0.01

In [218]:
#5. seqence 길이 단위 자르기

# 데이터 구성을 위한 리스트
x_data = []
y_data = []

## 문제(2): 반복문 내에서의 인덱싱을 사용하여 sequence_length 값 단위로 샘플을 잘라 데이터 만들기, y_str은 x_str은 한 칸씩 쉬프트된 sequnce

for i in range(0, len(sentence) - sequence_length):
  x_str = sentence[i : i + sequence_length]
  y_str = sentence[i+1 : i+1 + sequence_length]
  print(i, x_str, "->", y_str)

  # x_str과 y_str이 문자집합에 해당하는 인덱스를 각각 x_data, y_data에 append
  x_data.append([vocab[c] for c in x_str])
  y_data.append([vocab[d] for d in y_str])

0 Brick wall -> rick walls
1 rick walls -> ick walls 
2 ick walls  -> ck walls a
3 ck walls a -> k walls ar
4 k walls ar ->  walls are
5  walls are -> walls are 
6 walls are  -> alls are t
7 alls are t -> lls are th
8 lls are th -> ls are the
9 ls are the -> s are ther
10 s are ther ->  are there
11  are there -> are there 
12 are there  -> re there f
13 re there f -> e there fo
14 e there fo ->  there for
15  there for -> there for 
16 there for  -> here for a
17 here for a -> ere for a 
18 ere for a  -> re for a r
19 re for a r -> e for a re
20 e for a re ->  for a rea
21  for a rea -> for a reas
22 for a reas -> or a reaso
23 or a reaso -> r a reason
24 r a reason ->  a reason 
25  a reason  -> a reason a
26 a reason a ->  reason an
27  reason an -> reason and
28 reason and -> eason and 
29 eason and  -> ason and y
30 ason and y -> son and yo
31 son and yo -> on and you
32 on and you -> n and you 
33 n and you  ->  and you m
34  and you m -> and you mu
35 and you mu -> nd you mus
36

In [219]:
# 출력해서 한 칸씩 쉬프트된 것 확인하기!

print(x_data[0])
print(y_data[0])

[19, 11, 24, 9, 4, 5, 10, 12, 6, 6]
[11, 24, 9, 4, 5, 10, 12, 6, 6, 22]


In [220]:
##6. 입력 시퀀스에 대해 원핫인코딩 수행

## 문제(4) : x_data를 원핫인코딩 > numpy의 eye를 쓸 수 있지 않을까?
x_one_hot = [np.eye(vocab_size)[x] for x in x_data]

##7. 입력 데이터, 레이블데이터 텐서로 변환
x_one_hot = np.array(x_one_hot)
## 문제(5) : x_one_hot과 y_data 텐서로 변환 : 둘 다 같은 형식의 텐서로 변환하면 될까?? (FloatTensor, LongTesor 중 맞는 것은?)
X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)

In [221]:
##8. 크기 확인
print('훈련 데이터의 크기 : {}'.format(X.shape))
print('레이블의 크기 : {}'.format(Y.shape))

훈련 데이터의 크기 : torch.Size([188, 10, 26])
레이블의 크기 : torch.Size([188, 10])


In [222]:
##9.원핫인코딩 결과 샘플 확인하기
print(X[0])

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0.,

In [223]:
##10. 레이블 데이터 샘플 확인하기
print(Y[0])

tensor([11, 24,  9,  4,  5, 10, 12,  6,  6, 22])


In [235]:
##11. RNN 모델 구현

##문제(6) : 기본 pytorch 인자 넣기 연습 + forward 채우기
### 조건 : rnn layer 2개 쌓기 + 마지막은 fc layer
### batch_first 설정 필요할까? (유튜브 강의 참고)
'''
batch_first=True : 입력 데이터의 shape이 (batch_size, sequence_length, input_size) 순서.
batch_first=False : 입력 데이터의 shape이 (sequence_length, batch_size, input_size) 순서.
'''

class Net(torch.nn.Module):
  def __init__(self, input_dim, hidden_dim, layers):
    super(Net, self).__init__()
    self.rnn = torch.nn.RNN(input_size=input_dim, hidden_size=hidden_dim, num_layers=layers, batch_first=True)
    self.fc = torch.nn.Linear(in_features=hidden_dim, out_features=vocab_size)

  def forward(self, x):
    x, _ = self.rnn(x)
    x = self.fc(x)
    return x

In [236]:
net = Net(vocab_size, hidden_size, 2)

In [237]:
##12. loss function
criterion = torch.nn.CrossEntropyLoss()

##13. optimizer
optimizer = optim.Adam(net.parameters(), learning_rate)

In [238]:
##14. 출력 크기 점검
outputs = net(X)
print(outputs.shape)

torch.Size([188, 10, 26])


In [239]:
##15. Training 시작

for i in range(100):
    optimizer.zero_grad()
    outputs = net(X)
    ##문제(7) : outputs, Y 형태 그대로 넣으면 안되죠. view 함수를 이용해 loss값을 계산해봅시다.
    loss = criterion(outputs.view(-1, vocab_size), Y.view(-1))
    loss.backward()
    optimizer.step()

    #16. 예측결과 확인
    results = outputs.argmax(dim=2)
    predict_str = ""
    for j, result in enumerate(results):
        if j == 0: # 처음에는 예측 결과를 전부 가져오기
            predict_str += ''.join([world_set[t] for t in result])
        else: # 그 다음에는 마지막 글자만 반복 추가
            predict_str += world_set[result[-1]]

    print(predict_str)

yyyyyyyyyyyyyryyyyyyyyyyyyyrryyyryyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyryyyyyyyyyyyyyyyyyyyyyrysyyyyyyyyyyyyyyyyyyyyyrrryyyyyyyyyyyyyyyyyyyyyyyyryyyyyyyyyyyyyryysyryryyyyyyyyyyryyyyyyyyyyy
rr trt rttttt tt rr tttrttttrtttrttttttr tttrr trtttrrrttt rr trtrtt tttt rttttt ttt trrr tt ttttttt rr t trt  trtttt ttrt rrtrrrtt tt trrrtt tttt rttttt tt rr tt tttrrttrrrrrttrtttrt rt rtttrrrtrt
r                                                                                                                                                                                                    
r                                                                                                                                                                                                    
r                                                                                                                                                                                                    
          

In [230]:
predict_str

"rick walls are there tor a reason and you must not think that the brick walls are 't there to seep us out, but rather in this way that the brick walls are there to show us how badly we want thinksc"

결과가 어떤가요?? 마지막 에폭의 문장이 그럴싸한가요?

## Task2

위 sentence는 제가 임의로 생성한 문장들입니다.

마음에 드시는 문구 가져오셔서 문장이 어떻게 생성되는지 확인해보세요!

영어가 아닌 한국어로 시도해보는 것도 좋겠죠?

수정이 많이 필요(토큰화 등) 할 수 있으나 한번 시도해보시는 것 권장드립니다 :)

위 베이스라인은 어디든 수정하셔도 좋고 조금 더 자연스러운 문장이 나올 수 있게 다양한 시도를 해보세요!

조건 : 문장 3개 이상, 연결성이 있는 문장을 " " 으로 구분하여 ( )에 넣기

# [1. 모델1]
* sentence 소문자로 변경
* sequence_length=15, learning_rate=0.01
* RNN layers : 3

In [29]:
#1. 생성할 문장 데이터
sentence = ("Amidst the towering skyscrapers, a lone figure stands, "
            "gazing at the city lights that shimmer like distant stars. "
            "The city's heartbeat echoes through the streets, a rhythmic pulse ")
sentence = sentence.lower()
#2. 문자 집합 만들기
world_set = list(set(sentence))
vocab = {chr:idx for idx, chr in enumerate(world_set)}

#3. 문자 집합 크기 확인
vocab_size = len(vocab)
print('문자 집합 크기 : {}'.format(vocab_size))

문자 집합 크기 : 26


In [30]:
#4. 하이퍼파라미터 설정
hidden_size = vocab_size # 같아야 하는 것 확인!
sequence_length = 15  # 너무 길거나 너무 짧게 잡으면 안됩니다!
learning_rate = 0.01


#5. seqence 길이 단위 자르기
# 데이터 구성을 위한 리스트
x_data = []
y_data = []

for i in range(0, len(sentence) - sequence_length):
  x_str = sentence[i : i + sequence_length]
  y_str = sentence[i+1 : i+1 + sequence_length]
  print(i, x_str, "->", y_str)
  x_data.append([vocab[c] for c in x_str])
  y_data.append([vocab[d] for d in y_str])
print(x_data[0])
print(y_data[0])

0 amidst the towe -> midst the tower
1 midst the tower -> idst the toweri
2 idst the toweri -> dst the towerin
3 dst the towerin -> st the towering
4 st the towering -> t the towering 
5 t the towering  ->  the towering s
6  the towering s -> the towering sk
7 the towering sk -> he towering sky
8 he towering sky -> e towering skys
9 e towering skys ->  towering skysc
10  towering skysc -> towering skyscr
11 towering skyscr -> owering skyscra
12 owering skyscra -> wering skyscrap
13 wering skyscrap -> ering skyscrape
14 ering skyscrape -> ring skyscraper
15 ring skyscraper -> ing skyscrapers
16 ing skyscrapers -> ng skyscrapers,
17 ng skyscrapers, -> g skyscrapers, 
18 g skyscrapers,  ->  skyscrapers, a
19  skyscrapers, a -> skyscrapers, a 
20 skyscrapers, a  -> kyscrapers, a l
21 kyscrapers, a l -> yscrapers, a lo
22 yscrapers, a lo -> scrapers, a lon
23 scrapers, a lon -> crapers, a lone
24 crapers, a lone -> rapers, a lone 
25 rapers, a lone  -> apers, a lone f
26 apers, a lone f -> 

In [31]:
##6. 입력 시퀀스에 대해 원핫인코딩 수행

## 문제(4) : x_data를 원핫인코딩 > numpy의 eye를 쓸 수 있지 않을까?
x_one_hot = [np.eye(vocab_size)[x] for x in x_data]

##7. 입력 데이터, 레이블데이터 텐서로 변환
x_one_hot = np.array(x_one_hot)
X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)

##8. 크기 확인
print('훈련 데이터의 크기 : {}'.format(X.shape))
print('레이블의 크기 : {}'.format(Y.shape))

훈련 데이터의 크기 : torch.Size([165, 15, 26])
레이블의 크기 : torch.Size([165, 15])


In [35]:
class Net(torch.nn.Module):
  def __init__(self, input_dim, hidden_dim, layers):
    super(Net, self).__init__()
    self.rnn = torch.nn.RNN(input_size=input_dim, hidden_size=hidden_dim, num_layers=layers, batch_first=True)
    self.fc = torch.nn.Linear(in_features=hidden_dim, out_features=vocab_size)

  def forward(self, x):
    x, _ = self.rnn(x)
    x = self.fc(x)
    return x

net = Net(vocab_size, hidden_size, 3)
##12. loss function
criterion = torch.nn.CrossEntropyLoss()

##13. optimizer
optimizer = optim.Adam(net.parameters(), learning_rate)

##15. Training 시작

for i in range(100):
    optimizer.zero_grad()
    outputs = net(X)
    loss = criterion(outputs.view(-1, vocab_size), Y.view(-1))
    loss.backward()
    optimizer.step()

    #16. 예측결과 확인
    results = outputs.argmax(dim=2)
    predict_str = ""
    for j, result in enumerate(results):
        if j == 0: # 처음에는 예측 결과를 전부 가져오기
            predict_str += ''.join([world_set[t] for t in result])
        else: # 그 다음에는 마지막 글자만 반복 추가
            predict_str += world_set[result[-1]]

    print(predict_str)
print()
print(sentence)

,yeylllyylllll,,lly,l,,yy,yll,lylylyyl,lylly,,lll,l,lylyylll,lyllylllyllleyll,lll,yylll,eyllellll,lylll,lylllyely,ry,lylllely,lyllll,llyl,llll,y,yll,lllllll,ylllylyl,lll,llll,lll,
ttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttt
ttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttt
ttt     t  t t tt     t  ttt t   t t t  t   t t  t t          t t t  t   tt t      t t  t t t t t tt   t     t        t      t tt   t t  tt     t t    t  t t t    t t t t  t t t  
                                                                                                                                                                                   
                                                                                                    

# [2. 모델2]
* sequence_length=15, learning_rate=0.01, epoch=200
* optimizer를 AdamW 로 변경
* RNN layers : 4

In [38]:
hidden_size = vocab_size # 같아야 하는 것 확인!
sequence_length = 15  # 너무 길거나 너무 짧게 잡으면 안됩니다!
learning_rate = 0.01

class Net(torch.nn.Module):
  def __init__(self, input_dim, hidden_dim, layers):
    super(Net, self).__init__()
    self.rnn = torch.nn.RNN(input_size=input_dim, hidden_size=hidden_dim, num_layers=layers, batch_first=True)
    self.fc = torch.nn.Linear(in_features=hidden_dim, out_features=vocab_size)

  def forward(self, x):
    x, _ = self.rnn(x)
    x = self.fc(x)
    return x

net = Net(vocab_size, hidden_size, 4)
##12. loss function
criterion = torch.nn.CrossEntropyLoss()

##13. optimizer
optimizer = optim.AdamW(net.parameters(), learning_rate)

##15. Training 시작

for i in range(200):
    optimizer.zero_grad()
    outputs = net(X)
    loss = criterion(outputs.view(-1, vocab_size), Y.view(-1))
    loss.backward()
    optimizer.step()

    #16. 예측결과 확인

    results = outputs.argmax(dim=2)
    predict_str = ""
    for j, result in enumerate(results):
        if j == 0: # 처음에는 예측 결과를 전부 가져오기
            predict_str += ''.join([world_set[t] for t in result])
        else: # 그 다음에는 마지막 글자만 반복 추가
            predict_str += world_set[result[-1]]

    print(predict_str)
print()
print(sentence)

cgiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiii
cy                                                                                                                                                                                 
tt         t        t                   t  t   t     t        t      t   t                t   t            t          t                              tt   t                   t    
ttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttt
ttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttt
tttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttt

# [3. 모델3]
* FC layers 2개로 늘림, relu 추가.
* dropout=0.2 로 설정
* RNN layers : 2
* learning_rate=0.025

In [82]:
import torch.nn.functional as F

hidden_size = vocab_size # 같아야 하는 것 확인!
sequence_length = 10  # 너무 길거나 너무 짧게 잡으면 안됩니다!
learning_rate = 0.025

class Net(torch.nn.Module):
  def __init__(self, input_dim, hidden_dim, layers):
    super(Net, self).__init__()
    self.rnn = torch.nn.RNN(input_size=input_dim, hidden_size=hidden_dim, num_layers=layers, batch_first=True, dropout=0.2)
    self.fc1 = torch.nn.Linear(in_features=hidden_dim, out_features=hidden_dim)
    self.fc2 = torch.nn.Linear(in_features=hidden_dim, out_features=vocab_size)

  def forward(self, x):
    x, _ = self.rnn(x)
    x = F.relu(self.fc1(x))
    x = self.fc2(x)
    return x

net = Net(vocab_size, hidden_size, 2)
##12. loss function
criterion = torch.nn.CrossEntropyLoss()

##13. optimizer
optimizer = optim.AdamW(net.parameters(), learning_rate)

##15. Training 시작

for i in range(100):
    optimizer.zero_grad()
    outputs = net(X)
    loss = criterion(outputs.view(-1, vocab_size), Y.view(-1))
    print(loss)
    loss.backward()
    optimizer.step()

    #16. 예측결과 확인

    results = outputs.argmax(dim=2)
    predict_str = ""
    for j, result in enumerate(results):
        if j == 0: # 처음에는 예측 결과를 전부 가져오기
            predict_str += ''.join([world_set[t] for t in result])
        else: # 그 다음에는 마지막 글자만 반복 추가
            predict_str += world_set[result[-1]]

    print(predict_str)
print()
print(sentence)

tensor(3.2572, grad_fn=<NllLossBackward0>)
hhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhhh
tensor(2.9965, grad_fn=<NllLossBackward0>)
hhhh h      h   h   h  hh h     hhh     h    h       hh      h        h   h     h               hhhh     h     h h    h     hh        h h    h   h         h      h     h h h      
tensor(2.8945, grad_fn=<NllLossBackward0>)
                                                                                                                                                                                   
tensor(2.8390, grad_fn=<NllLossBackward0>)
ttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttt
tensor(2.8485, grad_fn=<NllLossBackward0>)
teeeeeteeeeeeeeeeetteeteeeteeeteteeeeeteeeeeeeteetteeeteeeeteteee

# [4. SGD]
* opimizer = SGD 로 설정
* 처음에는 잘 학습되지 않는다고 생각했는데, 그게 아니라 단지 속도가 매우 느렸음
* epoch = 10000으로 주고 100 에폭마다 출력. 2000 에폭은 넘어가야 말이 되는 문장을 출력.
* sequence_length=15, learning_rate = 0.05 로 설정

In [94]:
import torch.nn.functional as F

hidden_size = vocab_size # 같아야 하는 것 확인!
sequence_length = 15  # 너무 길거나 너무 짧게 잡으면 안됩니다!
learning_rate = 0.05

class Net(torch.nn.Module):
  def __init__(self, input_dim, hidden_dim, layers):
    super(Net, self).__init__()
    self.rnn = torch.nn.RNN(input_size=input_dim, hidden_size=hidden_dim, num_layers=layers, batch_first=True, dropout=0.2)
    self.fc = torch.nn.Linear(in_features=hidden_dim, out_features=vocab_size)

  def forward(self, x):
    x, _ = self.rnn(x)
    x = self.fc(x)
    return x

net = Net(vocab_size, hidden_size, 2)
##12. loss function
criterion = torch.nn.CrossEntropyLoss()

##13. optimizer
optimizer = optim.SGD(net.parameters(), learning_rate)

##15. Training 시작

for i in range(10000):
    optimizer.zero_grad()
    outputs = net(X)
    loss = criterion(outputs.view(-1, vocab_size), Y.view(-1))
    loss.backward()
    optimizer.step()

    #16. 예측결과 확인
    if i%100 == 0:
      results = outputs.argmax(dim=2)
      predict_str = ""
      for j, result in enumerate(results):
          if j == 0: # 처음에는 예측 결과를 전부 가져오기
              predict_str += ''.join([world_set[t] for t in result])
          else: # 그 다음에는 마지막 글자만 반복 추가
              predict_str += world_set[result[-1]]

      print(predict_str)
print()
print(sentence)

rkkkkkzkkzkkzkkzkkkkkbkkkzkkfzkzzkkkk.kzkkkfkkkkkkkzkzkkzkkkkkkkzkkkkbkkrzkkkkkkkkkkkzzbkkbkkkkkbkkzkkkkkkkkkzzkzzkkk.zkkrzkzkkekkkkbkkkekkkkfkbkzzkkzkbk.kkkkkkkbzkkkkzzkzkk.kbzkk
                                                                                                                                                                                   
                                                                                                                                                                                   
                   t            t t           t              t  t                    t                    t      t                    t                           t                
  tt  t   t t   t  t            t t    t      t    t  t    t t  t   t t    t    t    t  t    t t  t       t      t   t      t     e   t      t       t   t        t t        t     
  t   t e t t   te t e          t t te t t    t  t t  t t tt tt t e t t  t t    the  t  t    t t  t 